In [1]:
import numpy as np
import networkx as nx
import kmapper as km
import sklearn
import warnings
import matplotlib.pyplot as plt
import signal
import time
import random
from networkx.algorithms.similarity import graph_edit_distance

warnings.filterwarnings("ignore")

In [2]:
G1 = nx.star_graph(5)
G2 = nx.star_graph(5)
distance = graph_edit_distance(G1, G2)
distance

0.0

In [3]:
def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0 
    
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [4]:
def calculate_similarity(graph1,graph2):
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)
    
    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2) #k are different between the two graphs, then use the smaller one.
    similarity = sum((laplacian1[:k] - laplacian2[:k])**2) #sum of the squared differences between the largest k eigenvalues
    return similarity


In [5]:
def extract_graph_features(graph):
    pr = nx.pagerank(graph,0.9)
    dc = nx.degree_centrality(graph)
    cc = nx.closeness_centrality(graph)
    bx = nx.betweenness_centrality(graph)
    c = nx.clustering(graph)
    
    #create list for each features
    pr_list =  [i for i in pr.values()]
    dc_list =  [i for i in dc.values()]
    cc_list =  [i for i in cc.values()]
    bx_list =  [i for i in bx.values()]
    c_list =  [i for i in c.values()]
    d_list = [val for (node, val) in graph.degree()]
    data = np.column_stack((pr_list,dc_list,cc_list,bx_list,c_list,d_list))
    return data

In [16]:
def TDA_transformation(data):
    Xfilt = data
    mapper = km.KeplerMapper()
    scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1))
    Xfilt = scaler.fit_transform(Xfilt)
    lens = mapper.fit_transform(Xfilt, projection=sklearn.manifold.TSNE())
    cls = 2  # We use cls= 5

    graph = mapper.map(lens,Xfilt,clusterer=sklearn.cluster.KMeans(n_clusters=cls,random_state=1618033),
        cover=km.Cover(n_cubes=2, perc_overlap=0.7))
    return km.to_nx(graph)

In [39]:
def add_node_to_graph(graph,p):
    new_node = graph.number_of_nodes() + 1
    graph.add_node(new_node)
    existing_nodes = list(graph.nodes())[:-1]  # Exclude the new node
    for existing_node in existing_nodes:
        if random.random() < p:  
            graph.add_edge(new_node, existing_node)
    return graph
    

In [24]:
def remove_least_degree_node(graph):
    degrees = graph.degree()
    min_degree_node = min(degrees, key=lambda x: x[1])[0]
    # Remove the node with the minimum degree
    graph.remove_node(min_degree_node)
    return graph

In [52]:
def random_add_new_edge(graph):
    node_num = graph.number_of_nodes()
    node1 = random.randint(1,node_num)
    node2 = random.randint(1,node_num)
    while graph.has_edge(node1,node2) or node1 == node2 :
        node1 = random.randint(1,node_num)
        node2 = random.randint(1,node_num)
    graph.add_edge(node1,node2)
    return graph

In [46]:
def random_remove_edge(graph):
    random_edge = random.choice(list(graph.edges()))
    # Remove the randomly selected edge
    graph.remove_edge(*random_edge)
    return graph

In [88]:
def graph_generator(original_graph, node, edge,p):
    new_graph = original_graph.copy()
    if node < 0:
        for i in range(abs(node)):
            new_graph = remove_least_degree_node(new_graph)
    elif node > 0:
        for i in range(abs(node)):
            new_graph = add_node_to_graph(new_graph,p)
    
    if edge < 0:
        for i in range(abs(edge)):
            new_graph = random_remove_edge(new_graph)
    elif edge >0:
        for i in range(abs(edge)):
            new_graph = random_add_new_edge(new_graph)
    return new_graph

In [108]:
G = nx.star_graph(5)
G1 = graph_generator(G,node = 5,edge = 0,p=0.5)


graph_edit_distance(G,G1)





25.0

In [112]:
edge = G1.number_of_edges()
G2 = graph_generator(G1,node = -10,edge = 0,p = 0.5)
graph_edit_distance(G1,G2)
edge

25

In [15]:
hop = 2

for i in range(-hop,hop + 1):
    for j in range(-hop,hop + 1):
        if (i == -hop or i == hop or j == -hop or j == hop):
            print(i,j)
        

-2 -2
-2 -1
-2 0
-2 1
-2 2
-1 -2
-1 2
0 -2
0 2
1 -2
1 2
2 -2
2 -1
2 0
2 1
2 2
